In [20]:
import pandas as pd
from pprint import pprint
def pd_print_entry(df, index):
    entry = df.iloc[index]
    print("Sentence: ")
    print(entry['sentence'])
    print("Questions: ")
    print(entry['questions'])
    print('Answers')
    # print(entry['answers'])
    for answer in entry['answers']:
        print(answer)
    print('Removed')
    # print(entry['removed'])
    for removed in entry['removed']:
        print(removed)
    print('Phrases: ')
    pprint(entry['phrases'])

df = pd.read_csv('parsed_sentences.csv')

In [21]:
pd_print_entry(df, 0)

Sentence: 
In the case current file handle represents an object of type NFS4DIR, NFS4ERR _ISDIR is returned.
Questions: 
['What current file handle represents an object of type NFS4DIR?']
Answers
[
'
<
n
p
>
t
h
e
 
c
a
s
e
 
c
u
r
r
e
n
t
 
f
i
l
e
 
h
a
n
d
l
e
 
r
e
p
r
e
s
e
n
t
s
 
a
n
 
o
b
j
e
c
t
 
o
f
 
t
y
p
e
 
N
F
S
4
D
I
R
<
n
p
>
 
<
v
p
>
i
s
 
r
e
t
u
r
n
e
d
<
v
p
>
'
,
 
'
<
n
p
>
t
y
p
e
 
N
F
S
4
D
I
R
<
n
p
>
 
<
v
p
>
r
e
p
r
e
s
e
n
t
s
 
a
n
 
o
b
j
e
c
t
 
o
f
 
t
y
p
e
 
N
F
S
4
D
I
R
<
v
p
>
'
,
 
'
<
n
p
>
t
y
p
e
 
N
F
S
4
D
I
R
<
n
p
>
 
<
v
p
>
i
s
 
r
e
t
u
r
n
e
d
<
v
p
>
'
]
Removed
[
'
<
n
p
>
t
h
e
 
c
a
s
e
 
c
u
r
r
e
n
t
 
f
i
l
e
 
h
a
n
d
l
e
 
r
e
p
r
e
s
e
n
t
s
 
a
n
 
o
b
j
e
c
t
 
o
f
 
t
y
p
e
 
N
F
S
4
D
I
R
<
n
p
>
 
<
v
p
>
r
e
p
r
e
s
e
n
t
s
 
a
n
 
o
b
j
e
c
t
 
o
f
 
t
y
p
e
 
N
F
S
4
D
I
R
<
v
p
>
'
,
 
'
<
n
p
>
t
h
e
 
c
a
s
e
 
c
u
r
r
e
n
t
 
f
i
l
e
 
h
a
n
d
l
e
 
r
e
p
r
e
s
e
n
t
s
 
a
n
 
o
b
j
e
c
t
 
o
f
 
t
y
p
e
 
N
F
